In [ ]:
#Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os


In [ ]:
#DEFINE FILEPATH
hosp='W:\\Main_project\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\'
icu='W:\\Main_project\\mimic-iv-2.2\\mimic-iv-2.2\\icu\\'
base='W:\\Main_project\\base_files\\'

## Read main files

In [ ]:

## Read files
#Files required
#patients,admissions,procedures,labevents(subset),diagnoses,chartevents,icustays,d_diagnoses,d_proc,d_labitems
#hosp

patients=pd.read_csv(f'{hosp}patients.csv')
admissions=pd.read_csv(f'{hosp}admissions.csv')
#omr=pd.read_csv(f'{hosp}omr.csv')
procedures=pd.read_csv(f'{hosp}procedures_icd.csv')
d_procedures=pd.read_csv(f'{hosp}d_icd_procedures.csv')
diagnoses=pd.read_csv(f'{hosp}diagnoses_icd.csv')
d_diagnoses=pd.read_csv(f'{hosp}d_icd_diagnoses.csv')
labitems=pd.read_csv(f'{hosp}d_labitems.csv')
prescription=pd.read_csv(f'{hosp}prescriptions.csv')
#icu folder
icustays=pd.read_csv(f'{icu}icustays.csv')

In [ ]:
## Functions to read large files
def read_large_file(file,path):
    filename=pd.read_csv(f'{path}{file}',chunksize=10000)
    chunk_list = []
    for chunk in labs:    
        chunk_list.append(chunk)
        filename = pd.concat(chunk_list)
    return filename

#Function to get creatinine labs
def filter_scr_in_chunks_and_save(file_path, subject_ids, item_id=50912, chunksize=100000000):
    # Create a directory to store filtered chunks
    save_dir = os.path.dirname(file_path)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Iterate over the file in chunks
    chunk_index = 1
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        # Filter rows based on subject_ids
        filtered_chunk = chunk[chunk['subject_id'].isin(subject_ids)]
        
        # Filter rows based on item_id
        filtered_chunk = filtered_chunk[filtered_chunk['itemid'] == item_id]
        
        # Save the filtered chunk to a separate CSV file
        save_path = os.path.join(save_dir, f'chunk{chunk_index}_scr.csv')
        filtered_chunk.to_csv(save_path, index=False)
        
        chunk_index += 1


#Function to read only top rows to examine a file
def read_top_nrows(file,path,n_rows):
    filename=pd.read_csv(f'{path}{file}',nrows=n_rows)
    return filename


#####################

def map_valuenum(admissions_df, values_df):
    # Convert admittime and dischtime to datetime objects
    admissions_df['admittime'] = pd.to_datetime(admissions_df['admittime'])
    admissions_df['dischtime'] = pd.to_datetime(admissions_df['dischtime'])
    
    # Convert charttime to datetime object
    values_df['charttime'] = pd.to_datetime(values_df['charttime'])
    
    # Initialize an empty list to store mapped values
    mapped_values = []
    
    # Iterate through each row in admissions_df
    for index, row in admissions_df.iterrows():
        subject_id = row['subject_id']
        admittime = row['admittime']
        dischtime = row['dischtime']
        
        # Filter values_df for rows within the admission time window for the subject_id
        filtered_values = values_df[(values_df['subject_id'] == subject_id) & 
                                    (values_df['charttime'] >= admittime) &
                                    (values_df['charttime'] <= dischtime)]
        
        if not filtered_values.empty:
            # Find the row with the earliest charttime
            earliest_row = filtered_values.loc[filtered_values['charttime'].idxmin()]
            
            # Extract valuenum and charttime
            valuenum = earliest_row['valuenum']
            charttime = earliest_row['charttime']
            
            # Append the mapped values to the list
            mapped_values.append({'subject_id': subject_id, 
                                  'admittime': admittime,
                                  'dischtime': dischtime,
                                  'earliest_charttime': charttime,
                                  'earliest_valuenum': valuenum})
    
    # Create a DataFrame from the mapped values
    mapped_df = pd.DataFrame(mapped_values)
    
    return mapped_df


#Calculate egfr based on creatinine values

def calculate_egfr(row):
    if row['gender'] == 'F':
        gender_factor = 0.742
    else:
        gender_factor = 1

    if 'BLACK' in row['race'].upper():
        race_factor = 1.212
    else:
        race_factor = 1

    # Adding checks for zero division and null values
    if row['earliest_creatinine'] is not None and row['age'] is not None \
            and row['earliest_creatinine'] > 0 and row['age'] > 0:
        egfr = 175 * (row['earliest_creatinine'] ** -1.154) * (row['age'] ** -0.203) * gender_factor * race_factor
        return egfr
    else:
        # Return a default value or handle the error as appropriate
        return None  # Or any other value or action you prefer

## Counts

In [ ]:
print("No of patients",patients['subject_id'].nunique())
print("No of admissions",admissions['hadm_id'].nunique())

## Get latest encounter for a patient

In [ ]:
#first get the latest encounter for each patient and then filter
pat_adm=pd.merge(patients,admissions,how='inner',on='subject_id')
filter_df=pat_adm.groupby('subject_id')['admittime'].max().reset_index() #max admittime
pat_adm1 = pd.merge(pat_adm, filter_df, on=['subject_id', 'admittime'], how='inner')

In [ ]:
#pat_adm1[pat_adm1['subject_id'].isin([10000032,10000068,10000084])] #sanity check
#pat_adm[pat_adm['subject_id'].isin([10000032,10000068,10000084])] #sanity check
del filter_df

## Filter criteria -Age,hospital stay,dialysis,eGFR

In [ ]:
## Filter criteria

#Age

pat_adm1['birth_year'] = pat_adm1['anchor_year'] - pat_adm1['anchor_age']
pat_adm1['admittime'] = pd.to_datetime(pat_adm1['admittime'])
pat_adm1['anchor_year'] = pd.to_datetime(pat_adm1['anchor_year'], format='%Y')
pat_adm1['age'] = (pat_adm1['admittime'] - pd.to_datetime(pat_adm1['anchor_year'].dt.year, format='%Y')) / pd.Timedelta(days=365) + pat_adm1['anchor_age']
pat_adm1['age'] = pat_adm1['age'].astype(int)

print("Initial number of patients",pat_adm['subject_id'].nunique()) # 180733 patients initally
pat_adm1=pat_adm1[pat_adm1['age']>=18]
print("After age conditions",pat_adm1['subject_id'].nunique())


#Hospital stay for >= 3 days

pat_adm1['admittime'] = pd.to_datetime(pat_adm1['admittime'], format='%Y-%m-%d %H:%M:%S')
pat_adm1['dischtime'] = pd.to_datetime(pat_adm1['dischtime'], format='%Y-%m-%d %H:%M:%S')
pat_adm1['date_diff'] = (pat_adm1['dischtime'] - pat_adm1['admittime']).dt.days
pat_adm1 = pat_adm1[pat_adm1['date_diff'] >= 3]
print("After number of days of hospital stay conditions",pat_adm1['subject_id'].nunique())



#Did not undergo dialysis within 2 days of admission
dialysis_codes=('5A1D00Z','5A1D60Z','5A1D70Z','5A1D80Z','5A1D90Z','3993','3995','5498')
pat_proc= pd.merge(pat_adm1,procedures,on=['hadm_id','subject_id'],how='left')
pat_proc['admittime'] = pd.to_datetime(pat_proc['admittime'], format='%Y-%m-%d')
pat_proc['chartdate'] = pd.to_datetime(pat_proc['chartdate'], format='%Y-%m-%d')
pat_proc['dialysis_diff']=(pat_proc['chartdate'] - pat_proc['admittime'].dt.floor('d')).dt.days
pat_proc = pat_proc[pat_proc['dialysis_diff'] <= 2]
#pat_proc contains patients who underwent dialysis within 2 days after admission ; hence remove these hadm_ids from pat_adm
pat_adm1=pat_adm1[~pat_adm1['hadm_id'].isin(pat_proc['hadm_id'])]
print("After dialysis within 2 days conditions",pat_adm1['subject_id'].nunique())


### Below code get the creatinine values to get egfr and then filter patients with egfr<=15

In [ ]:
# Save the creatinine labs file
subject_ids=pat_adm['subject_id'].unique()  # Example set of subject_ids
file_path=f'{hosp}\\labevents.csv'  # Provide the path to your CSV file
filter_scr_in_chunks_and_save(file_path, subject_ids)


#read the creatinine lab files into a df
directory = f'{hosp}'
dfs = []
for filename in os.listdir(directory):
    if filename.startswith('chunk') and filename.endswith('scr.csv'):
        filepath = os.path.join(directory, filename)
        # Read the CSV file into a DataFrame and append it to the list
        dfs.append(pd.read_csv(filepath))
# Concatenate all DataFrames in the list into a single DataFrame
scr_df = pd.concat(dfs, ignore_index=True)
scr_df.to_csv(f'{hosp}scr_labs.csv') # so that above code need not be run all the time




#Map the scr values to patients
scr_map = map_valuenum(pat_adm1, scr_df)
scr_map.rename(columns={'earliest_valuenum': 'earliest_creatinine'}, inplace=True)
#join to main df
pat_adm1=pd.merge(pat_adm1,scr_map,on=['subject_id','admittime','dischtime'],how='left')


### Caluclate egfr and remove patients with <=15
pat_adm1['egfr'] = pat_adm1.apply(calculate_egfr, axis=1)
pat_adm1 = pat_adm1[pat_adm1['egfr'] > 15]
print("After removing patients with egfr<=15",pat_adm1['subject_id'].nunique())
#Save the file
pat_adm1.to_csv(f'{hosp}pat_adm_latest_encounter_filtered.csv')

## Calculate if AKI occured

In [2]:
### Criteria
#An increase in serum creatinine by greater than or equal to 0.3 mg/dL within 48 hours; or
#An increase in serum creatinine by greater than or equal to 1.5 times baseline, 
#which is known or presumed to have occurred within the prior 7 days; or

In [ ]:
# are there any icustays which are not in admissions file? its not there, so we are good
icustays[~icustays['hadm_id'].isin(admissions['hadm_id'])]['hadm_id'] # none

In [ ]:
#Read the patient and creatinine lab files, which was saved earlier from above code

pat_adm1=pd.read_csv(f'{base}pat_adm_latest_encounter_filtered.csv',index_col=0)
scr_df=pd.read_csv(f'{hosp}scr_labs.csv',index_col=0)
#Take only required columns
pat_adm1=pat_adm1[['subject_id', 'gender','hadm_id', 'admittime', 'dischtime','marital_status', 'race','age','earliest_creatinine', 'egfr']]
scr_df=scr_df[['labevent_id', 'subject_id','itemid','charttime','valuenum','valueuom']]

In [ ]:
#Merge df and convert the date columns
df=pd.merge(pat_adm1,scr_df,how='left',on='subject_id')
df=df[(df['charttime'] >= df['admittime']) & (df['charttime'] <= df['dischtime'])]
df['admittime'] = pd.to_datetime(df['admittime'],  format='%Y-%m-%d %H:%M:%S')
df['dischtime'] = pd.to_datetime(df['dischtime'],  format='%Y-%m-%d %H:%M:%S')
df['charttime'] = pd.to_datetime(df['charttime'],  format='%Y-%m-%d %H:%M:%S')

### Function to check 48 hour increase

In [ ]:
def check_increase(row, df):
    current_time = row['charttime']
    previous_measurements = df[(df['subject_id'] == row['subject_id']) & (df['charttime'] < current_time)]
    window_start = current_time - timedelta(hours=48)
    previous_measurements = previous_measurements[(previous_measurements['charttime'] >= window_start) & (previous_measurements['charttime'] < current_time)]
    if len(previous_measurements) > 0:
        min_value = previous_measurements['valuenum'].min()      
        if round(row['valuenum'] - min_value,2) >= 0.3:
            return 1, row['charttime'] #previous_measurements['charttime'].iloc[-1]
    return 0, None

In [ ]:
# Apply the function to each row
df[['increase_flag', 'start_time_of_increase']] = df.apply(lambda row: pd.Series(check_increase(row, df)), axis=1)
#save the file
df.to_csv(f'{base}48h_final.csv',index=False)

In [ ]:
#Check counts
print(df[df['increase_flag']==0]['subject_id'].nunique()) #27111
print(df[df['increase_flag']==1]['subject_id'].nunique()) #5269

In [ ]:
#test for one
#df[df['subject_id']==19779215]

In [ ]:
#Get only one row per subject, with charttime
df['start_time_of_increase'] = pd.to_datetime(df['start_time_of_increase'])

# Group by 'subject_id' and aggregate
result_df = df.groupby('subject_id').agg(
    increase_flag=('increase_flag', 'max'),
    start_time_of_increase=('start_time_of_increase', 'min')
).reset_index()

print(result_df[result_df['increase_flag']==0]['subject_id'].nunique()) #27111
print(result_df[result_df['increase_flag']==1]['subject_id'].nunique()) #5269
#Save the file
result_df.to_csv(f'{base}48h_aki_pid_time_final.csv',index=False)

### 7 day condition

In [ ]:
#Consider only patients with aki flag as 0, from previous 48h  criteria
subject_ids7=result_df[result_df['increase_flag']==0]['subject_id'].unique()
df_process=df.copy()
df_process=df_process[df_process['subject_id'].isin(subject_ids7)]


# Calculate baseline_scr and baseline_charttime
baseline_info = df_process.groupby('subject_id').apply(lambda x: x.loc[x['valuenum'].idxmin()]).reset_index(drop=True)
baseline_info = baseline_info[['subject_id', 'valuenum', 'charttime']]
baseline_info.columns = ['subject_id', 'baseline_scr', 'baseline_charttime']
df_process = pd.merge(df_process,baseline_info, on='subject_id', how='left')

#Calculate flag and charttime
df_process['increase_flag'] = 0
df_process['start_time_of_increase'] = None

for index, row in df_process.iterrows():
    if row['charttime'] > row['baseline_charttime'] and row['charttime'] <= row['baseline_charttime'] + timedelta(days=7):
        if row['valuenum'] >= 1.5 * row['baseline_scr']:
            df_process.at[index, 'increase_flag'] = 1
            df_process.at[index, 'start_time_of_increase'] = row['charttime']
            
#Save the file            
df_process.to_csv(f'{base}7D_final.csv',index=False)            

In [ ]:
#test data 
#df_process[df_process['subject_id']==10119391]

In [ ]:
# Get onle one record per subject with flag and start time

df_process['start_time_of_increase'] = pd.to_datetime(df_process['start_time_of_increase'])
# Group by 'subject_id' and aggregate
result_df = df_process.groupby('subject_id').agg(
    increase_flag=('increase_flag', 'max'),
    start_time_of_increase=('start_time_of_increase', 'min')
).reset_index()

#test 
#result_df[result_df['subject_id']==10131647]

print(result_df[result_df['increase_flag']==1]['subject_id'].nunique()) #520
print(result_df[result_df['increase_flag']==0]['subject_id'].nunique()) #21322

#Save the file
result_df.to_csv(f'{base}7D_aki_pid_time_final.csv',index=False)

### Combine data from 48 hour and 7 day conditions

In [ ]:
# files required for next steps :
aki7=pd.read_csv(f'{base}7D_aki_pid_time_final.csv',index_col=0)
aki7=aki7.reset_index()

aki48=pd.read_csv(f'{base}48h_aki_pid_time_final.csv',index_col=0)
aki48=aki48.reset_index()
to_be_removed_from48=aki48[aki48['increase_flag']==0]['subject_id']

aki48=aki48[~aki48['subject_id'].isin(to_be_removed_from48)] #retain only 1s in 48h


#Concat
aki_pid_time=pd.concat([aki48,aki7], ignore_index=True)


#Since charttime was given as min start time, get 24 hour prior 
aki_pid_time['start_time_of_increase']=pd.to_datetime(aki_pid_time['start_time_of_increase'])
#below code didn twork; so correct it when taking labs and medications
aki_pid_time['min_time_required'] = aki_pid_time['start_time_of_increase'] - timedelta(hours=24) if pd.notnull(aki_pid_time['start_time_of_increase']).all() else aki_pid_time['start_time_of_increase']

In [ ]:
#Merge dataframe
pat_adm1=pd.merge(pat_adm1,aki_pid_time,how='left',on='subject_id')

# Check if 'increase flag' is 0 and 'min_time_required' is null
# Assign 'dischtime' as 'min_time_required' for rows where the condition is true
mask = (pat_adm1['increase_flag'] == 0.0) & pat_adm1['min_time_required'].isnull()
pat_adm1.loc[mask, 'min_time_required'] = pat_adm1.loc[mask, 'dischtime']

In [ ]:
#Save the final file
pat_adm1.to_csv(f'{base}pat_adm_aki_time_final.csv',index=False)

### Final file - f'{base}pat_adm_aki_time_final.csv'